<a href="https://colab.research.google.com/github/hr1588/deep-learning/blob/main/NLP_2%EC%9E%A5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install ratsnlp -q

     |████████████████████████████████| 42 kB 551 kB/s 
     |████████████████████████████████| 2.8 MB 8.3 MB/s 
     |████████████████████████████████| 57 kB 3.7 MB/s 
     |████████████████████████████████| 582 kB 53.8 MB/s 
     |████████████████████████████████| 529 kB 46.3 MB/s 
     |████████████████████████████████| 3.3 MB 9.3 MB/s 
     |████████████████████████████████| 880 kB 48.7 MB/s 
     |████████████████████████████████| 163 kB 46.3 MB/s 
     |████████████████████████████████| 96 kB 4.6 MB/s 


In [3]:
!pip install transformers -q

### NSMC 로드(말뭉치 내려받기 및 전처리)

In [4]:
from Korpora import Korpora # 패키지 로드
nsmc = Korpora.load("nsmc", force_download=True)


    Korpora 는 다른 분들이 연구 목적으로 공유해주신 말뭉치들을
    손쉽게 다운로드, 사용할 수 있는 기능만을 제공합니다.

    말뭉치들을 공유해 주신 분들에게 감사드리며, 각 말뭉치 별 설명과 라이센스를 공유 드립니다.
    해당 말뭉치에 대해 자세히 알고 싶으신 분은 아래의 description 을 참고,
    해당 말뭉치를 연구/상용의 목적으로 이용하실 때에는 아래의 라이센스를 참고해 주시기 바랍니다.

    # Description
    Author : e9t@github
    Repository : https://github.com/e9t/nsmc
    References : www.lucypark.kr/docs/2015-pyconkr/#39

    Naver sentiment movie corpus v1.0
    This is a movie review dataset in the Korean language.
    Reviews were scraped from Naver Movies.

    The dataset construction is based on the method noted in
    [Large movie review dataset][^1] from Maas et al., 2011.

    [^1]: http://ai.stanford.edu/~amaas/data/sentiment/

    # License
    CC0 1.0 Universal (CC0 1.0) Public Domain Dedication
    Details in https://creativecommons.org/publicdomain/zero/1.0/



[nsmc] download ratings_train.txt: 14.6MB [00:00, 87.9MB/s]                            
[nsmc] download ratings_test.txt: 4.90MB [00:00, 32.2MB/s]                           


In [8]:
import os

# nsmc 전처리

def write_lines(path, lines):
    with open(path, 'w', encoding = 'UTF-8') as f:
        for line in lines:
            f.write(f'{line}\n')

write_lines("drive/MyDrive/Bert/train.txt", nsmc.train.get_all_texts())
write_lines("drive/MyDrive/Bert/test.txt", nsmc.test.get_all_texts())

- NSMC에 포함된 영화 리뷰들을 순수 텍스트 형태로 코랩 환경에 저장

### GPT 토크나이저 구축

- GPT 계열 모델이 사용하는 토크나이저 기법은 BPE
- BPE(바이트 페어 인코딩) : 데이터에서 가장 많이 등장한 문자열을 병합해서 데이터를 압축하는 기법, 빈도를 기준으로 병합

In [9]:
import os

os.makedirs("bbpe", exist_ok=True) # 저장 디렉토리 만들기

In [11]:
from tokenizers import ByteLevelBPETokenizer

bytebpe_tokenizer = ByteLevelBPETokenizer()

bytebpe_tokenizer.train(
    files = ["drive/MyDrive/Bert/train.txt", "drive/MyDrive/Bert/test.txt"], # 학습 말뭉치 리스트 형태로 삽입
    vocab_size = 10000, # 어휘 집합 크기 조절
    special_tokens = ["[PAD]"] # 특수 토큰 추가
)

bytebpe_tokenizer.save_model("bbpe") 

['bbpe/vocab.json', 'bbpe/merges.txt']

- json 파일 : 바이트 수준 BPE의 어휘 집합
- txt 파일 : 바이그램 쌍의 병합 우선 순위
- 바이그램 : 토큰을 2개씩 묶어서 나열한 것


BPE 토큰화 수행 방식

1. BPE는 어절별로 병합 우선순위가 높은 바이그램 쌍을 반복해서 병합
2. 병합된 토큰이 어휘 집합에 있는지 확인해 최종 결과 도출

### Bert 토크나이저 구축

- BERT는 워드피스 토크나이저를 사용
- 워드피스(wordpiece) : 말뭉치에서 자주 등장한 문자열을 토큰으로 인식한다는 점이 BPE와 본질적으로 유사하지만, 어휘 집합을 구축할 때 문자열을 병합하는 기준이 다름
- 워드패스는 병합했을 때 말뭉치의 우도를 가장 높히는 쌍을 병합
- 우도(likelihood) : 고정된 관측값이 어떠한 확률분포에서 어느정도의 확률로 나타나는지에 대한 확률(확률밀도함수가 나타내는 그래프의 y값)

In [13]:
import os

os.makedirs("wordpiece", exist_ok=True)

In [16]:
from tokenizers import BertWordPieceTokenizer

wordpiece_tokenizer = BertWordPieceTokenizer(lowercase = False)
wordpiece_tokenizer.train(
    files = ["drive/MyDrive/Bert/train.txt","drive/MyDrive/Bert/test.txt"],
    vocab_size = 10000
)

wordpiece_tokenizer.save_model("wordpiece")

['wordpiece/vocab.txt']

워드피스 토큰화 수행 방식

1. 분석 대상 어절에 어휘 집합에 있는 서브워드가 포함되어 있을 때 해당 서브워드를 어절에서 분리

2. 어절의 나머지에서 어휘 집합에 있는 서브워드 탐색(최장 일치 기준) 후 분리

- 이 때, 분석 대상 문자열에서 서브워드 후보가 하나도 없으면 해당 문자열 전체를 미등록 단어 취급

### GPT 입력값 만들기

In [17]:
from transformers import GPT2Tokenizer

tokenizer_gpt = GPT2Tokenizer.from_pretrained("bbpe")
tokenizer_gpt.pad_token = "[PAD]"

file bbpe/config.json not found


- 입력값을 만들기 위해 BPE 어휘 집합과 바이그램 병합 우선순위 필요

In [18]:
sentences = [
    "아 더빙.. 진짜 짜증나네요 목소리",
    "흠.. 포스터보고 초딩영화줄,,, 오버연기조차 가볍지 않구나",
    "심금을 울리는 영화다",
    "둘이서 보다가 하나가 죽어도 모르겠다",
    "이제껏 이런 성향의 영화를 본 적이 없다 "
]

tokenized_sentences = [tokenizer_gpt.tokenize(sentence) for sentence in sentences]
print(tokenized_sentences)

[['ìķĦ', 'ĠëįĶë¹Ļ', '..', 'Ġì§Ħì§ľ', 'Ġì§ľì¦ĿëĤĺ', 'ëĦ¤ìļĶ', 'Ġëª©ìĨĮë¦¬'], ['íĿł', '..', 'Ġíı¬ìĬ¤íĦ°', 'ë³´ê³ł', 'Ġì´ĪëĶ©', 'ìĺģíĻĶ', 'ì¤Ħ', ',,,', 'Ġìĺ¤ë²Ħ', 'ìĹ°ê¸°', 'ì¡°ì°¨', 'Ġê°Ģë³į', 'ì§Ģ', 'ĠìķĬ', 'êµ¬ëĤĺ'], ['ìĭ¬', 'ê¸Ī', 'ìĿĦ', 'Ġìļ¸ë¦¬ëĬĶ', 'ĠìĺģíĻĶëĭ¤'], ['ëĳĺ', 'ìĿ´', 'ìĦľ', 'Ġë³´ëĭ¤ê°Ģ', 'ĠíķĺëĤĺê°Ģ', 'Ġì£½', 'ìĸ´ëıĦ', 'Ġëª¨ë¥´ê²łëĭ¤'], ['ìĿ´ìłľ', 'ê»ı', 'ĠìĿ´ëŁ°', 'ĠìĦ±', 'íĸ¥', 'ìĿĺ', 'ĠìĺģíĻĶë¥¼', 'Ġë³¸', 'Ġìłģ', 'ìĿ´', 'ĠìĹĨëĭ¤', 'Ġ']]


In [19]:
batch_input = tokenizer_gpt(
    sentences,
    padding = "max_length", # 문장의 최대 길이에 맞춰 패딩
    max_length = 12,        # 문장의 토큰 기준 최대 길이
    truncation = True       # 문장 잘림 허용 여부
)

# 코드의 실행 결과로 input_ids, attention_mask 입력값 제작

In [20]:
batch_input["input_ids"] 

[[334, 2338, 263, 581, 4055, 464, 3808, 0, 0, 0, 0, 0],
 [3693, 263, 2720, 758, 2883, 356, 806, 1907, 4444, 875, 2960, 7292],
 [669, 616, 313, 6992, 1200, 0, 0, 0, 0, 0, 0, 0],
 [3518, 264, 318, 1876, 9347, 916, 1553, 2132, 0, 0, 0, 0],
 [2310, 3289, 649, 963, 1588, 306, 777, 626, 1284, 264, 897, 221]]

- 토큰화 결과를 가지고 각 토큰을 인덱스로 바꾼 것, 어휘 집합을 통해 확인할 수 있음
- 인덱싱 : 각 토큰을 인덱스로 변환하는 과정

In [21]:
batch_input["attention_mask"] # attention_mask : 일반 토큰이 자리한 곳과 패딩 토큰이 자리한 곳을 구분해 알려주는 장치

[[1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0],
 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 [1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0],
 [1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0],
 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]

### Bert 토큰화

In [22]:
from transformers import BertTokenizer

tokenizer_bert = BertTokenizer.from_pretrained(
    "wordpiece", do_lower_case = False
)

file wordpiece/config.json not found


In [23]:
tokenized_sentences_bert = [tokenizer_bert.tokenize(sentence) for sentence in sentences]

batch_inputs = tokenizer_bert(
    sentences,
    padding = "max_length",
    max_length = 12,
    truncation = True
)

# 코드의 실행결과로 3가지 입력값 제작

In [24]:
batch_inputs["input_ids"]

[[2, 621, 2631, 16, 16, 1993, 3678, 1990, 3323, 3, 0, 0],
 [2, 997, 16, 16, 2609, 2045, 2796, 1981, 1115, 14, 14, 3],
 [2, 601, 1087, 1031, 6440, 2195, 3, 0, 0, 0, 0, 0],
 [2, 5014, 1105, 2382, 8357, 8422, 1037, 2693, 3, 0, 0, 0],
 [2, 2233, 1624, 1999, 564, 1317, 1044, 2038, 503, 9035, 2080, 3]]

- CLS : 모든 문장 앞에 대응하는 인덱스(2)
- SEP : 모든 문장 뒤에 대응하는 인덱스(3)
- Bert는 문장 시작과 끝에 이 2개 토큰을 덧붙임

In [25]:
batch_inputs['attention_mask']

[[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0],
 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 [1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0],
 [1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0],
 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]

In [26]:
batch_inputs["token_type_ids"]

[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]

- token_type_ids : 세그먼트(segment)에 해당하는 입력값
- BERT 모델은 기본적으로 문서(문장) 2개를 입력받는데, 이 때 token_type_ids로 구분
- 첫 번째 세그먼트에 해당하는 입력값은 0, 두 번째 세그먼트는 1
- 해당 코드에서는 문장을 하나씩 넣었기 때문에 모든 값이 0